# TODO: Title
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [2]:
# TODO: Install any packages that you might need

In [ ]:
# TODO: Import any packages that you might need

## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

In [ ]:
import os
import json
import boto3
from tqdm import tqdm

import json
from tqdm import tqdm

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

download_and_arrange_data()

100%|██████████| 1228/1228 [01:59<00:00, 10.30it/s]


100%|██████████| 2299/2299 [03:50<00:00,  9.97it/s]


100%|██████████| 2666/2666 [04:27<00:00,  9.98it/s]


100%|██████████| 2373/2373 [03:57<00:00,  9.99it/s]


100%|██████████| 1875/1875 [03:01<00:00, 10.32it/s]


In [15]:
import os
import boto3
import sagemaker
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs, ProfilerConfig, FrameworkProfile
from sagemaker.analytics import HyperparameterTuningJobAnalytics
from PIL import Image
import shutil

In [16]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [17]:
print(f"Default Bucket: {bucket}")
print(f"RoleArn: {role}")

Default Bucket: sagemaker-us-east-1-482545180177
RoleArn: arn:aws:iam::482545180177:role/service-role/AmazonSageMaker-ExecutionRole-20230729T112806


## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).

In [18]:
#TODO: Perform any data cleaning or data preprocessing
from os import listdir
import os.path

count = 0
dataset_folder = 'train_data'

for file in os.listdir(dataset_folder):
    path = os.path.join(dataset_folder, file)
    number = len(os.listdir(path))
    print(f'Number of images with the label of "{file}": {number}')
    count += number
print(f'\nNumber of images in total: {count}')
total_images = count

Number of images with the label of "4": 2373
Number of images with the label of "2": 2299
Number of images with the label of "1": 1228
Number of images with the label of "5": 1875
Number of images with the label of "3": 2666

Number of images in total: 10441


In [19]:
# We are preparing our folders for train-test-validation split

train_test_val_split = ['train','validation','test']
labels =['1','2','3','4','5']

new_folder_dataset = 'amazon_images'

if not os.path.exists(new_folder_dataset):
    os.makedirs(new_folder_dataset)
    print(f'Successfully created a new folder named "{new_folder_dataset}"')

for folder in train_test_val_split:
    
    if not os.path.exists(os.path.join(new_folder_dataset, folder)):
        os.makedirs(os.path.join(new_folder_dataset, folder))
        print(f'Successfully created the path of "{os.path.join(new_folder_dataset, folder)}"')
        
    for label in labels:
        path = os.path.join(new_folder_dataset, folder, label)
        if not os.path.exists(path):
            os.makedirs(path)
            print(f'Successfully created the path of "{path}"')
            
# Useful method to delete a folder with files inside
# import shutil
# shutil.rmtree('bin_images')

In [20]:
train_ratio = 0.8
test_ratio = (1 - train_ratio)/2
validation_ratio = (1 - train_ratio)/2
    
for label in os.listdir(dataset_folder):
    
    path = os.path.join(dataset_folder, label)
    dataset = (os.listdir(path))
    dataset_len = len((os.listdir(path)))
    
    train_len = round(dataset_len * train_ratio)
    train_list = dataset[:train_len]
    
    test_len = round(dataset_len * test_ratio)
    test_list = dataset[train_len:train_len+test_len]
    
    validation_len = round(dataset_len * validation_ratio)
    validation_list = dataset[train_len+test_len:]
    
    list_dictionary = {'train': train_list,
                       'test': test_list,
                       'validation': validation_list}
    
    print("Label number is: " + label)
    
    for folder in train_test_val_split:
#         print('Resizing of the images is starting...')
        print(f'Saving the files to {new_folder_dataset}/{folder} folder\n')
        for file in list_dictionary[folder]:
            path = os.path.join(dataset_folder, label, file)
            if 'ipynb' not in path:
                
# This can be used to resize images in this step

#                 img = Image.open(path).convert('RGB')
#                 resized_img = img.resize((224,224),Image.BILINEAR)
#                 image_path = os.path.join(new_folder_dataset, folder, label, file)     
#                 resized_img.save(image_path)

                copy = os.path.join(dataset_folder, label, file)
                paste = os.path.join(new_folder_dataset, folder, label)
                shutil.copy(copy, paste)

print('\nSaving process has been completed.')

Label number is: 4
Saving the files to amazon_images/train folder

Saving the files to amazon_images/validation folder

Saving the files to amazon_images/test folder

Label number is: 2
Saving the files to amazon_images/train folder

Saving the files to amazon_images/validation folder

Saving the files to amazon_images/test folder

Label number is: 1
Saving the files to amazon_images/train folder

Saving the files to amazon_images/validation folder

Saving the files to amazon_images/test folder

Label number is: 5
Saving the files to amazon_images/train folder

Saving the files to amazon_images/validation folder

Saving the files to amazon_images/test folder

Label number is: 3
Saving the files to amazon_images/train folder

Saving the files to amazon_images/validation folder

Saving the files to amazon_images/test folder


Saving process has been completed.


In [21]:
bucket = sagemaker_session.default_bucket()
prefix = 'dataset_images'
# !aws s3 cp amazon_images s3://aws-udacity-capstone/ --recursive #alternative
session = sagemaker.session.Session()
inputs = session.upload_data(path='amazon_images', bucket=bucket, key_prefix=prefix)

In [22]:
print(f"input spec (in this case, just an S3 path): {inputs}")

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-482545180177/dataset_images


## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [39]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values


#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.01),
    "batch_size": CategoricalParameter([32, 64, 128]),
    "epochs": IntegerParameter(4, 6),
}

objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test Loss: ([+-]?[0-9\\.]+)"}]

In [40]:
#TODO: Create your training estimator
estimator = PyTorch(
    entry_point="train.py",
    role=role,
    py_version='py3',
    framework_version="1.8",
    instance_count=1,
    instance_type= "ml.g4dn.xlarge",
    # 'ml.m5.2xlarge',
    )

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=3,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [42]:
# TODO: Fit your estimator
tuner.fit({"training": inputs}, wait=True) # TODO: Remember to include your data channels

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


Using provided s3_resource
....................................................................................*


UnexpectedStatusException: Error for HyperParameterTuning job pytorch-training-230731-0907: Failed. Reason: All training jobs failed. Please take a look at the training jobs failures to get more details.

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
#TODO: Create your hyperparameter search space

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

In [ ]:
# TODO: Find the best hyperparameters

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
]

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

In [ ]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point="train_model.py",
    framework_version="1.6",
    py_version="py36",
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    hyperparameters=best_hyperparameters,
    rules=rules,
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
)

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Plot a debugging output.
estimator.fit({"training": inputs}, wait=True)

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
from sagemaker.pytorch import PyTorchModel

model_data = estimator.output_path + estimator.latest_training_job.job_name + "/output/model.tar.gz"
print(f"Model: {model_data}")

In [ ]:
# TODO: Run an prediction on the endpoint
from sagemaker.predictor import Predictor
jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()
# this class ImgPredictor is a subclass created from the Predictor class and is used to send image data to the endpoint
class ImgPredictor(Predictor):
    def __init__( self, endpoint_name, sagemaker_session):
        super(ImgPredictor, self).__init__(
            endpoint_name,
            sagemaker_session = sagemaker_session,
            serializer = jpeg_serializer,
            deserializer = json_deserializer
        )

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
# TODO: Deploy your model to an endpoint
pytorch_model = PyTorchModel(
    model_data=model_data, 
    role=role, 
    entry_point='inference.py',
    py_version="py36",
    framework_version="1.8",
    predictor_cls = ImgPredictor
)

predictor = pytorch_model.deploy(initial_instance_count=1, 
                                 instance_type="ml.m5.2xlarge")
# TODO: Add your deployment configuration like instance type and number of instances

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances